# Dataset

CIFAR 10

In [ ]:
import torchvision

cifar10Train = torchvision.datasets.CIFAR10("./CIFAR10", download=True, transform=lambda im: torchvision.transforms.functional.pil_to_tensor(im)/255)
cifar10Test = torchvision.datasets.CIFAR10("./CIFAR10", train=False, download=True, transform=lambda im: torchvision.transforms.functional.pil_to_tensor(im)/255)

Files already downloaded and verified


In [ ]:
import numpy as np
import pandas as pd
import time

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import PIL
from sklearn.metrics import accuracy_score, f1_score

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Models

## CNN5

3 Conv, 2 FC, Dropout=0.4, Batch norm, skip connections

Activation fn: GELU

Optimizer: Adam

In [ ]:
class CNN5(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn1 = nn.Conv2d(3, 16, (3, 3), padding='same').to(device)
        self.bn1 = nn.BatchNorm2d(16)
        self.gelu1 = nn.GELU()
        self.maxpool1 = nn.MaxPool2d((2, 2), stride=(2, 2)).to(device)

        self.cnn2 = nn.Conv2d(16, 32, (3, 3), padding='same').to(device)
        self.bn2 = nn.BatchNorm2d(32)
        self.gelu2 = nn.GELU()
        self.maxpool2 = nn.MaxPool2d((2, 2), stride=(2,2)).to(device)

        self.cnn3 = nn.Conv2d(32, 64, (3, 3), padding='same').to(device)
        self.bn3 = nn.BatchNorm2d(64)
        self.gelu3 = nn.GELU()
        self.maxpool3 = nn.MaxPool2d((2, 2), stride=(2,2)).to(device)

        self.linear1 = nn.Linear(64 * 4 * 4, 512)
        self.gelu4 = nn.GELU()
        self.dropout1 = nn.Dropout(0.4)

        self.linear2 = nn.Linear(512, 512)
        self.gelu5 = nn.GELU()
        self.dropout2 = nn.Dropout(0.4)

        self.linear3 = nn.Linear(512, 10)

    def forward(self, inputs):
        x1 = self.cnn1(inputs)
        x1 = self.bn1(x1)
        x1 = self.gelu1(x1)
        x1 += inputs
        x1 = self.maxpool1(x1)

        x2 = self.cnn2(x1)
        x2 = self.bn2(x2)
        x2 = self.gelu2(x2)
        x2 += x1
        x2 = self.maxpool2(x2)

        x3 = self.cnn3(x2)
        x3 = self.bn3(x3)
        x3 = self.gelu3(x3)
        x3 += x2
        x3 = self.maxpool3(x3)

        x = torch.flatten(x3, 1)

        x = self.linear1(x)
        x = self.gelu4(x)
        x = self.dropout1(x)

        x = self.linear2(x)
        x = self.gelu5(x)
        x = self.dropout2(x)

        x = self.linear3(x)
    
        return x


In [ ]:
trainloader = DataLoader(cifar10Train, 64)
testloader = DataLoader(cifar10Test, 64)

In [ ]:
def train_one_epoch(model, optimzer, loss_fn, training_loader):
    running_loss = 0.
    running_momentum = 0.

    for i, data in enumerate(training_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs.to(device))

        loss = loss_fn(outputs, labels.to(device))
        loss.backward()

        optimizer.step()
           

        running_loss += loss.item()

        
    average_loss = running_loss/(i+1)
    

    return average_loss

In [ ]:
def evaluate(tdataset, model):
    model.eval()
    preds = []
    truths = []
    for example in tdataset:
        input = example[0].unsqueeze(0).to(device)
        logits = model(input)
        pred = torch.argmax(torch.softmax(logits, 1))
        preds.append(pred.item())
        truths.append(example[1])
    return accuracy_score(truths, preds), f1_score(truths, preds, average='macro'), preds, truths

In [ ]:
def save_model(model, filepath):
    torch.save(model.state_dict(), filepath)

def load_model(model, filepath, device='cpu'):
    model.load_state_dict(torch.load(filepath, map_location=device))
    model.eval()

In [ ]:
cnn5 = CNN5().to(device)

optimizer = torch.optim.Adam(cnn5.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
EPOCHS = 25

total_time = 0

max_f1 = -float('inf')

log = []

for epoch in range(EPOCHS):
    begin = time.time()
    
    cnn5.train(True)
    average_train_loss = train_one_epoch(cnn5, optimizer, loss_fn, trainloader)
    
    end = time.time()
    total_time += end-begin

    cnn5.eval()
    running_test_loss = 0.
    for i, data in enumerate(testloader):
        inputs, labels = data
        outputs = cnn5(inputs.to(device))
        loss = loss_fn(outputs, labels.to(device))
        running_test_loss += loss.item()
    average_test_loss = running_test_loss/(i+1)
    end = time.time()
    if (epoch+1)%1 == 0:
        train_accuracy, train_f1, _, _ = evaluate(cifar10Train, cnn5)
        test_accuracy, test_f1, _, _ = evaluate(cifar10Test, cnn5)
        if test_f1 > max_f1:
            max_f1 = test_f1
            save_model(cnn5, './models/cnn5.pth')
        log.append({'average_test_loss': average_test_loss, 'average_train_loss': average_train_loss, 'total_time': total_time, 'train_accuracy': train_accuracy, 'train_f1': train_f1, 'test_accuracy': test_accuracy, 'test_f1': test_f1})
        print(f"Epoch {epoch+1} | train loss: {average_train_loss:.3f} | train accuracy: {100*train_accuracy:.2f}% | train f1: {train_f1:.2f} | test loss: {average_test_loss:.3f} | test accuracy: {100*test_accuracy:.2f}% | test f1: {test_f1:.2f} | time: {total_time:.2f}s")

In [ ]:
import pickle

pickle.dump(log, open('./logs/cnn5.pkl', 'wb'))